In [ ]:
import json
import requests
import pandas as pd
import datetime
import discord
from discord.ext import commands
import redis

TOKEN="<SECRET>"


r = redis.Redis(
  host= "<SECRET>",
  port= "<SECRET>",
  password= "<SECRET>"
)

# maintain a persistent database of tracked threads linked to redis
thread_dict = {}
for key in r.scan_iter():
    thread_dict[json.loads(key)] = json.loads(r.get(key))
    print(json.loads(key), json.loads(r.get(key)))

intents = discord.Intents.default()
intents.messages = True # enabling reading chat history on Guilds

bot = discord.Bot(intents=intents)


@bot.event
async def on_ready():
    # executes when the bot connects to each discord guild, making sure the tracked threads are aligned with the database
    for guild in bot.guilds:
        redis_connection.setnx(guild.id, json.dumps([]))
        if guild.id not in thread_dict.keys():
            thread_dict[guild.id] = []
        print(f"Bot {bot.user.name} is logged into guild {guild} (id: {guild.id})")

@bot.event
async def on_message(message):
    
    if message.author != bot.user:
        if message.channel.id in thread_dict[message.guild.id]:
            print("This message is in a tracked thread")
            data = {
                "threadId": message.channel.id,
                'author': message.author.name,
                'content': message.content,
                'guild': message.guild,
            }
            # creates a new message whenever there's an update in a tracked thread
            r = requests.post(url="https://terrarium-notion.fly.dev/page/update",
            json = json.loads(json.dumps(data, default=str)) # dumping and then loading deals with string issues etc.
            )

@bot.slash_command(description="Indicates the bot's latency.", guild_ids=['977651180308160512'])
async def ping(ctx):
    await ctx.respond(f"Pong! Latency is {round(bot.latency, 3)} seconds")


@bot.slash_command(description="Connect Discord threads to analytics", guild_ids=['977651180308160512'])
async def analyze(ctx):
    try:
        redis_connection = redis.Redis(
            host= "<SECRET>",
            port= "<SECRET>",
            password= "<SECRET>"
        )
        channel = ctx.channel
        print("CHANNEL TYPE:", channel.type)

        # check if we're in a thread
        if str(channel.type) == 'public_thread':
            await ctx.respond(f"Working with thread!")

            # add to tracked threads and database
            if channel.id not in thread_dict[ctx.guild.id]:
                thread_dict[ctx.guild.id].append(channel.id)
            print('UPDATED THREAD DICT', thread_dict)
            redis_connection.set(ctx.guild.id, json.dumps(thread_dict[ctx.guild.id]))

            messages = [{
                'author': message.author.name,
                'content': message.content,
                'clean_content': message.clean_content,
                'created_at': message.created_at,
                'jump_url': message.jump_url

            } async for message in channel.history(limit=200, oldest_first=True) if message.author != bot.user]
            df = pd.DataFrame(messages)

            # saves to csv for further testing and analys
            desktop = os.path.join(os.path.join(os.path.expanduser('~')), 'Desktop')
            df.to_csv(f"{desktop}/{ctx.channel.name}-data.csv", index=False, header=True)

            thread_info = {
                'id': channel.id,
                'identifier': f"{channel.id}-{str(hash(str(datetime.datetime.now())))[1:7]}",
                'created_at': channel.created_at,
                'jump_url': channel.jump_url,
                'name': channel.name,
                'taggedDev': None,


            }
            data = {
            "thread": thread_info,
            "messages": df.to_dict(orient='records'),
            "user": ctx.author.name,
            "discriminator": ctx.author.discriminator,
            "guild": ctx.guild.name

            }

            print("THREAD INFO:", thread_info)
            # sends thread data to the notion server
            r = requests.post(url="https://terrarium-notion.fly.dev/page/create",
            # r = requests.post(url="http://localhost:3001/page/create",
            json = json.loads(json.dumps(data, default=str)) # dumping and then loading deals with string issues etc.
            )
            print("STATUS", r.ok, r.status_code)
            res_json = r.json()

            print("RES_JSON", res_json)

            if r.ok:
                await ctx.send("Successfully added this thread to your Notion database")

            # handles errors
            if not r.ok:
                error_message = res_json["message"]
                if error_message == 'NoAnchor':
                    await ctx.send("I couldn't find an anchor page named 'Terrarium'. Have you shared it with the Notion bot?")
                elif error_message == "MultipleDB":
                    await ctx.send("There are multiple databases in your anchor page named 'Support Tickets'. Please delete one so I know which one to update!")
                elif error_message == "AxiosRequestError":
                    await ctx.send("There was an issue creating a Support Tickets database. This is probably related to Notion API Authentication.")
                elif error_message == "AddPageError":
                    await ctx.send("There was an issue adding this thread to the Supports Ticket database. Working on it!")
                elif error_message == "NotionAuthError":
                    await ctx.send("Somethin went wrong with Notion authentication. Make sure that this user and guild are authorized on the client app")
                elif error_message == "UnspecifiedError":
                    await ctx.send("Something went wrong, but I'm not sure what. Working on it!")
                else:
                    await ctx.send("Finn forgot to add some error condition. Bad Finn!")
            return r
        else:
            await ctx.send("/analyze can only be called from within a public thread")
    except Exception as e:
        print(e.response)
        print(bool(e.response))
        error_json = e.response.json()
        print(error_json)
        await ctx.send("Something went wrong.")

bot.run(TOKEN)